In [7]:
from baseline.utils import dataset, generate_test_splits, score
from nltk.corpus import stopwords
import html
import re
from nltk import sent_tokenize
import nltk.data
from nltk.tokenize import word_tokenize

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

import os
os.chdir('/home/kamal/CS249/Codes/sas/fnc/Codes/')

In [8]:
def preprocess(text):
    text = html.unescape(text)
    text = text.replace("\\n"," ")
    #text = text.replace("\"","")
    text = text.replace("_NEG","")
    text = text.replace("_NEGFIRST", "")
    #text = re.sub(r"#", "", text)
    text = re.sub(r"\*", "", text)
    #text = re.sub(r"\'s", "", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    #text = re.sub(r",", "", text)
    #text = re.sub(r"!", " !", text)
    #text = re.sub(r"\(", "", text)
    #text = re.sub(r"\)", "", text)
    #text = re.sub(r"\?", " ?", text)
    #text = re.sub(r'[^\x00-\x7F]',' ', text)
    #text = re.sub(r'[^\w\s]',' ',text)
    #text = re.sub("\d+", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    #text = text.rstrip(',|.|;|:|\'|\"')
    #text = text.strip('\'|\"|" "')
    #text=text.lstrip('" "')
    text=text.replace('..', '. ')
    text=text.replace('--', '. ')
    text = text.replace('\n','. ')

    return text

In [9]:
train_dataset = dataset.DataSet()
test_dataset = dataset.DataSet('test')

Reading dataset
Total stances: 49972
Total bodies: 1683
Reading dataset
Total stances: 25413
Total bodies: 904


In [10]:
data=dict()
for s in test_dataset.articles:
    a=list()
    text=test_dataset.articles[s]
    
    text=text.replace('”', '')
    text=text.replace('“', '')
    text=text.replace('"', '')
    text=text.replace('..', '. ')
    text=text.replace('--', '. ')
    text = text.replace('\n','. ')
    
    text=preprocess(text)
    sents=tokenizer.tokenize(text)
    if(len(sents)==1):
        sents=sents[0].split('.')
        
    for st in sents:
        st=preprocess(st)
        newsents=tokenizer.tokenize(st)
        for nst in newsents:
            if(len(nst)>1):
                splits=nst.split(' ')
                if(len(splits)>85):
                    a.append(' '.join(splits[:int(len(splits)/2)]))
                    a.append(' '.join(splits[int(len(splits)/2):]))
                    
                else:
                    a.append(nst)
            
    data[s]=a    

for s in train_dataset.articles:
    a=list()
    text=train_dataset.articles[s]
    
    text=text.replace('”', '')
    text=text.replace('“', '')
    text=text.replace('"', '')
    text=text.replace('..', '. ')
    text=text.replace('--', '. ')
    text = text.replace('\n','. ')
    
    text=preprocess(text)
    sents=tokenizer.tokenize(text)
    if(len(sents)==1):
        sents=sents[0].split('.')
    for st in sents:
        st=preprocess(st)
        newsents=tokenizer.tokenize(st)
        for nst in newsents:
            if(len(nst)>1):
                splits=nst.split(' ')
                if(len(splits)>85):
                    a.append(' '.join(splits[:int(len(splits)/2)]))
                    a.append(' '.join(splits[int(len(splits)/2):]))
                    
                else:
                    a.append(nst)
    data[s]=a    

In [6]:
import numpy as np
np.save('datavar.npy',data)

In [5]:
# Creating Pretrained Skipthought Embeddings
import os
os.chdir('/home/kamal/CS249/models/research/skip_thoughts')
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import os.path
import scipy.spatial.distance as sd
from skip_thoughts import configuration
from skip_thoughts import encoder_manager 


/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [6]:
VOCAB_FILE = "pretrained/skip_thoughts_uni_2017_02_02/vocab.txt"
EMBEDDING_MATRIX_FILE = "pretrained/skip_thoughts_uni_2017_02_02/embeddings.npy"
CHECKPOINT_PATH = "pretrained/skip_thoughts_uni_2017_02_02/model.ckpt-501424"

In [7]:
# Set up the encoder. Here we are using a single unidirectional model.
# To use a bidirectional model as well, call load_model() again with
# configuration.model_config(bidirectional_encoder=True) and paths to the
# bidirectional model's files. The encoder will use the concatenation of
# all loaded models.
encoder = encoder_manager.EncoderManager()
encoder.load_model(configuration.model_config(),
                   vocabulary_file=VOCAB_FILE,
                   embedding_matrix_file=EMBEDDING_MATRIX_FILE,
                   checkpoint_path=CHECKPOINT_PATH)


INFO:tensorflow:Reading vocabulary from pretrained/skip_thoughts_uni_2017_02_02/vocab.txt
INFO:tensorflow:Loaded vocabulary with 930914 words.
INFO:tensorflow:Loading embedding matrix from pretrained/skip_thoughts_uni_2017_02_02/embeddings.npy
INFO:tensorflow:Loaded embedding matrix with shape (930914, 620)
INFO:tensorflow:Building model.
Instructions for updating:
Please switch to tf.train.create_global_step
INFO:tensorflow:Loading model from checkpoint: pretrained/skip_thoughts_uni_2017_02_02/model.ckpt-501424
INFO:tensorflow:Restoring parameters from pretrained/skip_thoughts_uni_2017_02_02/model.ckpt-501424
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-501424


In [8]:
encoder.load_model(configuration.model_config(bidirectional_encoder=True),
                   vocabulary_file="pretrained/skip_thoughts_bi_2017_02_16/vocab.txt",
                   embedding_matrix_file="pretrained/skip_thoughts_bi_2017_02_16/embeddings.npy",
                   checkpoint_path="pretrained/skip_thoughts_bi_2017_02_16/model.ckpt-500008")

INFO:tensorflow:Reading vocabulary from pretrained/skip_thoughts_bi_2017_02_16/vocab.txt
INFO:tensorflow:Loaded vocabulary with 930914 words.
INFO:tensorflow:Loading embedding matrix from pretrained/skip_thoughts_bi_2017_02_16/embeddings.npy
INFO:tensorflow:Loaded embedding matrix with shape (930914, 620)
INFO:tensorflow:Building model.
INFO:tensorflow:Loading model from checkpoint: pretrained/skip_thoughts_bi_2017_02_16/model.ckpt-500008
INFO:tensorflow:Restoring parameters from pretrained/skip_thoughts_bi_2017_02_16/model.ckpt-500008
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-500008


In [5]:
len(data)

2587

In [30]:
data[2130][17]='junk text'
data[2130][18]='junk text'

In [31]:
for i in data:
    for j in range(0,len(data[i])):
        if(len(data[i][j].split(' '))>85):
            print(i,j)

In [73]:
body_encodings=dict()
count=0
for i in data:
    count+=1
    print(count,i)
    body_encodings[i] = encoder.encode(data[i])

1 1
2 2
3 3
4 12
5 19
6 20
7 21
8 28
9 31
10 33
11 37
12 38
13 39
14 42
15 46
16 49
17 50
18 52
19 57
20 58
21 64
22 66
23 67
24 69
25 71
26 73
27 79
28 85
29 90
30 92
31 94
32 96
33 97
34 98
35 100
36 101
37 102
38 103
39 106
40 111
41 113
42 122
43 123
44 127
45 128
46 129
47 131
48 133
49 138
50 141
51 143
52 145
53 150
54 153
55 157
56 160
57 163
58 164
59 165
60 166
61 172
62 174
63 177
64 178
65 180
66 182
67 183
68 184
69 191
70 192
71 196
72 197
73 201
74 202
75 207
76 211
77 215
78 216
79 222
80 223
81 231
82 233
83 237
84 238
85 245
86 246
87 248
88 249
89 250
90 252
91 253
92 256
93 257
94 259
95 261
96 262
97 264
98 268
99 270
100 274
101 275
102 278
103 279
104 284
105 287
106 290
107 291
108 293
109 294
110 297
111 301
112 306
113 309
114 310
115 312
116 314
117 315
118 316
119 317
120 318
121 319
122 325
123 326
124 327
125 328
126 329
127 332
128 334
129 335
130 336
131 338
132 340
133 341
134 351
135 353
136 354
137 356
138 358
139 360
140 361
141 363
142 369
143 378
1

982 118
983 119
984 120
985 121
986 124
987 125
988 126
989 130
990 132
991 134
992 135
993 136
994 137
995 139
996 140
997 142
998 144
999 146
1000 147
1001 148
1002 149
1003 151
1004 152
1005 154
1006 155
1007 156
1008 158
1009 159
1010 161
1011 162
1012 167
1013 168
1014 169
1015 170
1016 171
1017 173
1018 175
1019 176
1020 179
1021 181
1022 185
1023 186
1024 187
1025 188
1026 189
1027 190
1028 193
1029 194
1030 195
1031 198
1032 199
1033 200
1034 203
1035 204
1036 205
1037 206
1038 208
1039 209
1040 210
1041 212
1042 213
1043 214
1044 217
1045 218
1046 219
1047 220
1048 221
1049 224
1050 225
1051 226
1052 227
1053 228
1054 229
1055 230
1056 232
1057 234
1058 235
1059 236
1060 239
1061 240
1062 241
1063 242
1064 243
1065 244
1066 247
1067 251
1068 254
1069 255
1070 258
1071 260
1072 263
1073 265
1074 266
1075 267
1076 269
1077 271
1078 272
1079 273
1080 276
1081 277
1082 280
1083 281
1084 282
1085 283
1086 285
1087 286
1088 288
1089 289
1090 292
1091 295
1092 296
1093 298
1094 299
1

1862 1430
1863 1433
1864 1434
1865 1435
1866 1436
1867 1437
1868 1438
1869 1439
1870 1442
1871 1446
1872 1447
1873 1453
1874 1455
1875 1457
1876 1458
1877 1459
1878 1461
1879 1462
1880 1463
1881 1465
1882 1466
1883 1467
1884 1468
1885 1471
1886 1472
1887 1473
1888 1474
1889 1476
1890 1477
1891 1478
1892 1480
1893 1481
1894 1483
1895 1485
1896 1487
1897 1488
1898 1489
1899 1491
1900 1492
1901 1494
1902 1496
1903 1498
1904 1500
1905 1501
1906 1502
1907 1503
1908 1506
1909 1507
1910 1508
1911 1509
1912 1513
1913 1515
1914 1517
1915 1518
1916 1519
1917 1521
1918 1522
1919 1523
1920 1524
1921 1528
1922 1529
1923 1530
1924 1531
1925 1532
1926 1533
1927 1534
1928 1535
1929 1538
1930 1539
1931 1540
1932 1541
1933 1542
1934 1543
1935 1544
1936 1545
1937 1547
1938 1549
1939 1551
1940 1554
1941 1555
1942 1556
1943 1559
1944 1560
1945 1562
1946 1563
1947 1564
1948 1565
1949 1567
1950 1568
1951 1569
1952 1570
1953 1572
1954 1573
1955 1574
1956 1575
1957 1577
1958 1579
1959 1580
1960 1582
1961 1584


In [60]:
head_data=list()
ids=list()
for s in test_dataset.stances:
    if s['Headline'] not in ids:
        a=list()
        text=s['Headline']
        text=text.replace('”', '')
        text=text.replace('“', '')
        text=text.replace('"', '')
        text=preprocess(text)
        head_data.append(text)
        ids.append(s['Headline'])

for s in train_dataset.stances:
    if s['Headline'] not in ids:
        a=list()
        text=s['Headline']
        text=text.replace('”', '')
        text=text.replace('“', '')
        text=text.replace('"', '')
        head_data.append(text)
        ids.append(s['Headline']) 

In [61]:
print(len(head_data))
print(len(ids))

2532
2532


In [62]:
enc=encoder.encode(head_data)

In [63]:
ids[0]

'Ferguson riots: Pregnant woman loses eye after cops fire BEAN BAG round through car window'

In [64]:
head_encodings=dict()
count=0
for i in range(0,len(enc)):
    head_encodings[ids[i]] = enc[i]

In [38]:
length=0
for i in head_data:
    length=max(length,len(head_data[i].split(' ')))
length

41

In [59]:
len(head_encodings)

2532

In [43]:
encoder.encode(head_data['Crazy Conservatives Are Sure a Gitmo Detainee Killed James Foley'])

ValueError: Expected 0 < len(seq) <= 1, got 0

In [56]:
bigdata=list()
ids=list()
for s in test_dataset.articles:
    text=test_dataset.articles[s]
    text=text.replace('”', '')
    text=text.replace('“', '')
    text=text.replace('"', '')
    text=text.replace('..', '. ')
    text=text.replace('--', '. ')
    text = text.replace('\n','. ')
    text = text.replace('.','. ')
    
    text=preprocess(text)
    bigdata.append(text)
    ids.append(s)    

for s in train_dataset.articles:
    text=train_dataset.articles[s]
    text=text.replace('”', '')
    text=text.replace('“', '')
    text=text.replace('"', '')
    text=text.replace('..', '. ')
    text=text.replace('--', '. ')
    text = text.replace('\n','. ')
    text = text.replace('.','. ')
    
    text=preprocess(text)
    bigdata.append(text)
    ids.append(s)      
    

In [69]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

In [70]:
len(bigdata)

2587

In [77]:
l='sasank . is a.'
l.strip('.')

'sasank . is a'

In [49]:
bigdata=list()
ids=list()
for i in data:
    l='\n '.join(w for w in data[i])
    bigdata.append(l)
    ids.append(i)

In [50]:
j=0
for i in data:
    if(len(data[i])!=len(tokenizer.tokenize(bigdata[j]))):
        print(i,j)
    j+=1

411 149
926 314
1600 532
2555 872
2575 892
2578 895
200 1032
497 1221
645 1330
790 1430
1057 1608
1221 1716
1556 1941
1800 2095
2130 2315


In [44]:
j=0
for i in data:
    if(len(data[i])!=len(tokenizer.tokenize(bigdata[j]))):
        print(i,j)
        bigdata[j]=bigdata[j].replace('\n','.\n')
    j+=1

926 314
1600 532
2555 872
2575 892
2578 895
200 1032
497 1221
645 1330
790 1430
1057 1608
1556 1941
1800 2095
2130 2315


In [54]:
data[2555]

['Jonathan Swain and Amy Harder at Axios reported on Sunday morning that President Donald Trump has told confidants that he has made the decision to pull the United States out of the Paris Climate Accords, and will make an announcement soon, most likely this coming week.',
 'This report will certainly produce a major backlash from the anti-fossil fuel lobby in the U.S. and globally, which correctly views the commitments made by former President Barack Obama under this executive agreement as the main driving force for increasingly restrictive regulations of the U.S. fossil fuel industries.',
 "But this report, if true, should surprise no one after the President was the only G7 leader who refrained from endorsing the Paris Accords on Saturday, a move that came after one of his advisers had told the media that Mr. Trump's views related to Paris were evolving.",
 'This statement was taken by many Paris supporters as an indication that the President might be moving towards changing his mind

In [53]:
tokenizer.tokenize(bigdata[872])

['Jonathan Swain and Amy Harder at Axios reported on Sunday morning that President Donald Trump has told confidants that he has made the decision to pull the United States out of the Paris Climate Accords, and will make an announcement soon, most likely this coming week.',
 'This report will certainly produce a major backlash from the anti-fossil fuel lobby in the U.S. and globally, which correctly views the commitments made by former President Barack Obama under this executive agreement as the main driving force for increasingly restrictive regulations of the U.S. fossil fuel industries.',
 "But this report, if true, should surprise no one after the President was the only G7 leader who refrained from endorsing the Paris Accords on Saturday, a move that came after one of his advisers had told the media that Mr. Trump's views related to Paris were evolving.",
 'This statement was taken by many Paris supporters as an indication that the President might be moving towards changing his mind

In [ ]:
benc1=encoder.encode(bigdata[:int(len(bigdata)/2)])

In [ ]:
len(benc1)

In [ ]:
benc2=encoder.encode(bigdata[int(len(bigdata)/2):])

In [ ]:
len(benc2)

In [14]:
print('as ucla',repr(len(bigdata)))

as ucla 2587
